XGBoost 

In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.utils import resample
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, accuracy_score
from sentence_transformers import SentenceTransformer
import tqdm
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

#APPROACH 1

In [ ]:
input_path = r"C:\Users\Dell\Documents\final_processed_dataframe.pkl"

# Load the dataframe back
df = pd.read_pickle(input_path, compression="gzip")

print("DataFrame loaded successfully!")
print(df.shape)

In [ ]:
df.columns 

Index(['example_id', 'query', 'query_id', 'product_id', 'product_locale_x',
       'esci_label', 'small_version', 'large_version', 'split',
       'product_title', 'product_description', 'product_bullet_point',
       'product_brand', 'product_color', 'product_locale_y', 'product_text',
       'lemmatized_text'],
      dtype='object')

In [ ]:
# Compute class weights
classes = np.unique(df['esci_label'])
weights = compute_class_weight('balanced', classes=classes, y=df['esci_label'])
class_weights = dict(zip(classes, weights))
print(class_weights)

{'C': np.float64(8.625744995816438), 'E': np.float64(0.38420379989025877), 'I': np.float64(2.483354519276738), 'S': np.float64(1.1381723434503679)}


In [ ]:
# Stratified sampling to maintain class distribution
_, df_sample = train_test_split(df, test_size=0.15, stratify=df['esci_label'], random_state=42)
print(df_sample['esci_label'].value_counts(normalize=True))

esci_label
E    0.650697
S    0.219649
I    0.100670
C    0.028984
Name: proportion, dtype: float64


In [ ]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Batch encode (avoid RAM spikes)
X = model.encode(
    df_sample['lemmatized_text'].tolist(),
    batch_size=256,
    show_progress_bar=True
)

y = df_sample['esci_label']